<a href="https://colab.research.google.com/github/Chloe714/evalsamis/blob/main/Workshop3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# -----------------------------------
# STEP 1: Install Required Packages
# -----------------------------------
# These include LangChain, FAISS for vector search,
# PyPDF2 and pypdf for PDF parsing, pandas for CSVs,
# sentence-transformers for embeddings,
# and transformers for loading LLMs.
!pip install langchain faiss-cpu PyPDF2 pypdf pandas sentence-transformers transformers accelerate

In [9]:
# Install the missing package
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [10]:
# -----------------------------------
# STEP 2: Upload a PDF or CSV File
# -----------------------------------
# Allows user to upload a document file to work with.
from google.colab import files
uploaded = files.upload()

# Grab the uploaded file path
file_path = list(uploaded.keys())[0]
print(f"Uploaded file: {file_path}")


Saving Workshop3data.pdf to Workshop3data (1).pdf
Uploaded file: Workshop3data (1).pdf


In [11]:
# -----------------------------------
# STEP 3: Load and Parse the File
# -----------------------------------
# Use LangChain loaders to read either a PDF or CSV.
from langchain.document_loaders import PyPDFLoader, CSVLoader

if file_path.endswith(".pdf"):
    loader = PyPDFLoader(file_path)  # Parses PDFs into LangChain Documents
elif file_path.endswith(".csv"):
    loader = CSVLoader(file_path)    # Parses CSVs row-by-row into Documents
else:
    raise ValueError("Only PDF or CSV supported for this demo.")

documents = loader.load()
print(f"Loaded {len(documents)} documents.")


Loaded 1 documents.


In [12]:
# -----------------------------------
# STEP 4: Embed Documents with Hugging Face and Store in FAISS
# -----------------------------------
# Converts the loaded documents into vector embeddings,
# which are then stored in a FAISS index for fast similarity search.
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Use a compact embedding model from Hugging Face
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create a FAISS index from the embedded documents
db = FAISS.from_documents(documents, embedding)

# Set up a retriever to perform semantic search from the vector store
retriever = db.as_retriever()


/tmp/ipython-input-12-18449180.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [13]:
# -----------------------------------
# STEP 5: Load a Local LLM using Hugging Face Transformers
# -----------------------------------
# This loads a small instruction-tuned language model locally,
# and wraps it in a LangChain-compatible interface.
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from google.colab import userdata

# Choose a compact instruct-tuned model
model_id = "google/flan-t5-base"

# Load tokenizer and model (auto selects GPU if available)
# Use the HF_TOKEN secret for authentication
# HF_TOKEN = userdata.get("HF_TOKEN")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Set up a text generation pipeline
hf_pipeline = pipeline(
    "text2text-generation",  # For encoder-decoder models like T5
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,      # Limits length of the generated answer
    temperature=0.5,          # Controls randomness
)

# Wrap pipeline with LangChain LLM interface
llm = HuggingFacePipeline(pipeline=hf_pipeline)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-13-3998227145.py:29: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [14]:
# -----------------------------------
# STEP 6: Create a RetrievalQA Chain
# -----------------------------------
# Combines the retriever with the LLM to build a question-answering system.
from langchain.chains import RetrievalQA

# Create the QA chain using the local model and the FAISS retriever
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Example query
question = "What is this document about?"
result = qa_chain.run(question)
print("Answer:", result)


/tmp/ipython-input-14-3154872422.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(question)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer: Financial Analyst


In [ ]:
# -----------------------------------
# STEP 7: Ask More Questions (Interactive Mode)
# -----------------------------------
# Loop to let the user type custom queries and get answers from the document.
while True:
    query = input("Ask a question (or type 'exit'): ")
    if query.lower() == "exit":
        break
    print("Answer:", qa_chain.run(query))


Ask a question (or type 'exit'): hello how are you


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer: 555-555-0131
